# Proof of Concept
Ariel Silbert

In [235]:
import pandas as pd
import json
import os
import numpy as np
from sklearn.decomposition import PCA, IncrementalPCA
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import StandardScaler

## Load Data
- import json files
- turn into pandas dataframes
- split data into user and video information

In [29]:
#pd.read_json('TikTokImpact-main\TikTok_scrape\output\07032022_fresh\_.maur._\_.maur._.json')
#/Users/arielsilbert/Desktop/MS Data Analytics/Capstone/TikTokImpact-main/TikTok_scrape/output/07032022_fresh/_.maur._
#pd.read_json('_.maur._.json')
data = json.load(open('TikTokImpact-main/TikTok_scrape/output/07032022_fresh/_.maur._/_.maur._.json'))
df = pd.json_normalize(data)
df, data

(  itemList              user.id user.shortId user.uniqueId user.nickname  \
 0       []  6628114544067756038                   _.maur._          Maur   
 
                                    user.avatarLarger  \
 0  https://p16-sign-va.tiktokcdn.com/tos-maliva-a...   
 
                                    user.avatarMedium  \
 0  https://p16-sign-va.tiktokcdn.com/tos-maliva-a...   
 
                                     user.avatarThumb user.signature  \
 0  https://p16-sign-va.tiktokcdn.com/tos-maliva-a...           🌍🤝🇺🇦   
 
    user.createTime  ...  user.privateAccount user.secret  user.isADVirtual  \
 0       1548330054  ...                False       False             False   
 
    user.roomId  stats.followerCount  stats.followingCount  stats.heart  \
 0                              5336                     5      1400000   
 
    stats.heartCount  stats.videoCount  stats.diggCount  
 0           1400000                 3                0  
 
 [1 rows x 28 columns],
 {'user': {'

In [131]:
user_stats = pd.DataFrame()
video_stats = pd.DataFrame()

directory = 'TikTokImpact-main/TikTok_scrape/output'
for datefolder in os.listdir(directory): #go into folders by date
    f = os.path.join(directory, datefolder)
    if os.path.isdir(f): #checking if it is a directory
        for subfolder in os.listdir(f): #go into subfolders
            g = os.path.join(f, subfolder)
            if os.path.isdir(g):
                for json_file in os.listdir(g): #go into video and user stats folders
                    h = os.path.join(g, json_file)
                    if 'OrigSearch' in h:
                        data = json.load(open(h))
                        data['date'] = int(h.split('/')[3].split('_')[0]) #select date from folder name
                        df = pd.json_normalize(data)
                        video_stats = video_stats.append(df) #not an efficient way of loading the data to the df
                    else: 
                        data = json.load(open(h))
                        data['date'] = int(h.split('/')[3].split('_')[0])
                        df = pd.json_normalize(data)
                        user_stats = user_stats.append(df)

In [132]:
video_stats.iloc[:,:]
#numerical: createTime, itemCommentStatus, video.duration, video.bitrate, 
#music.duration, stats.diggCount, stats.shareCount, stats.commentCount, 
#stats.playCount, authorStats.followingCount, authorStats.followerCount, 
#authorStats.heartCount, authorStats.videoCount, authorStats.diggCount, authorStats.heart

,id,desc,createTime,challenges,originalItem,officalItem,textExtra,secret,forFriend,digged,...,stats.playCount,duetInfo.duetFromId,authorStats.followingCount,authorStats.followerCount,authorStats.heartCount,authorStats.videoCount,authorStats.diggCount,authorStats.heart,author.roomId,effectStickers
0,7069787462242962694,#ukraine #russia,1646063166,"[{'id': '451042', 'title': 'ukraine', 'desc': ...",False,False,"[{'awemeId': '', 'start': 0, 'end': 8, 'hashta...",False,False,False,...,6500000,0,0,163700,4600000,862,518,4600000,NaN,NaN
0,7068610977184369925,Russian invasion of Ukraine #ukraine #ukraina ...,1645789245,"[{'id': '451042', 'title': 'ukraine', 'desc': ...",False,False,"[{'awemeId': '', 'start': 28, 'end': 36, 'hash...",False,False,False,...,941,0,67,33,230,7,303,230,NaN,NaN
0,7068388771854634245,I wish this is a bad dream and we will wake up...,1645737509,"[{'id': '861413', 'title': 'stopwar', 'desc': ...",False,False,"[{'awemeId': '', 'start': 56, 'end': 64, 'hash...",False,False,False,...,14000000,0,70,115500,2500000,36,64,2500000,NaN,NaN
0,7071700963953741062,🇺🇦🤝 🇷🇺 #ukraine #supportukraine #fyp #nowar,1646508688,"[{'id': '451042', 'title': 'ukraine', 'desc': ...",False,False,"[{'awemeId': '', 'start': 12, 'end': 20, 'hash...",False,False,False,...,4600000,0,38,10600000,220500000,667,3338,220500000,NaN,NaN
0,7068781101208063275,President #Zelensky declared martial law after...,1645828854,"[{'id': '1651472006418438', 'title': 'zelensky...",False,False,"[{'awemeId': '', 'start': 10, 'end': 19, 'hash...",False,False,False,...,3100000,0,16,1700000,93100000,3186,704,93100000,7072286572741364526,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,7070523638717517102,Reporters shook by war in a “relatively civili...,1646234571,"[{'id': '229207', 'title': 'fyp', 'desc': '', ...",False,False,"[{'awemeId': '', 'start': 69, 'end': 73, 'hash...",False,False,False,...,13100000,0,42,3400000,60900000,517,12,60900000,NaN,NaN
0,7069128389994515718,pray for ukraine 🇺🇦❤️,1645909714,NaN,False,False,NaN,False,False,False,...,4600000,0,120,26500,733500,34,1404,733500,NaN,NaN
0,7071423972822519086,Ukraine VS Russia | Part 12,1646444196,NaN,False,False,NaN,False,False,False,...,16500000,0,1110,3600000,68600000,776,3459,68600000,NaN,NaN
0,7068774707562073349,слава україні🇺🇦 we have family there & in all ...,1645827366,"[{'id': '14927', 'title': 'russia', 'desc': 'Д...",False,False,"[{'awemeId': '', 'start': 110, 'end': 117, 'ha...",False,False,False,...,10300000,0,75,9696,2400000,161,3939,2400000,NaN,NaN


In [197]:
user_stats.fillna(0)
user_stats.iloc[:,20:]
#numerical columns: 'stats.followerCount', 'stats.followingCount', 'stats.heart', 'stats.heartCount', 'stats.videoCount', 'stats.diggCount'
#Emily to look into what createTime means IRL
#user.relation appears to be all 0s -- unclear what it is
#user.bioLink.risk probably a cybersecurity thing
#user...Setting seem to be T/F values but as numbers?
#diggs used to be likes -- field may not be used anymore, but still exists

,user.stitchSetting,user.privateAccount,user.secret,user.isADVirtual,user.roomId,stats.followerCount,stats.followingCount,stats.heart,stats.heartCount,stats.videoCount,stats.diggCount,user.commerceUserInfo.commerceUser,user.commerceUserInfo.category,user.commerceUserInfo.categoryButton,user.commerceUserInfo.downLoadLink.android,user.commerceUserInfo.downLoadLink.ios
0,0.0,False,False,False,,163700.0,0.0,4600000.0,4600000.0,862.0,0.0,NaN,NaN,NaN,NaN,NaN
0,3.0,False,False,False,,33.0,67.0,230.0,230.0,7.0,0.0,NaN,NaN,NaN,NaN,NaN
0,0.0,False,False,False,,115500.0,70.0,2500000.0,2500000.0,36.0,0.0,True,Personal Blog,False,NaN,NaN
0,0.0,False,False,False,,10600000.0,38.0,220500000.0,220500000.0,667.0,0.0,NaN,NaN,NaN,NaN,NaN
0,0.0,False,False,False,7072286572741364526,1700000.0,16.0,93100000.0,93100000.0,3186.0,0.0,True,Media,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.0,False,False,False,,3400000.0,42.0,60900000.0,60900000.0,517.0,0.0,True,Entertainment,False,NaN,NaN
0,0.0,False,False,False,,26500.0,120.0,733500.0,733500.0,34.0,0.0,NaN,NaN,NaN,NaN,NaN
0,3.0,False,False,False,,3600000.0,1110.0,68600000.0,68600000.0,776.0,0.0,NaN,NaN,NaN,NaN,NaN
0,3.0,False,False,False,,9696.0,75.0,2400000.0,2400000.0,161.0,0.0,NaN,NaN,NaN,NaN,NaN


## Preliminary Statistics
- correlations
- description (mean, std, count, min, max, 25th and 75th %iles)

In [198]:
video_stats.corr().dropna(how='all').dropna(axis=1).iloc[:,:13]

,createTime,itemCommentStatus,duetEnabled,stitchEnabled,date,video.height,video.width,video.duration,video.bitrate,author.verified,author.openFavorite,author.commentSetting,author.duetSetting
createTime,1.000000,0.020401,0.038341,0.018964,0.034735,0.051241,0.011305,0.003746,-0.044217,0.029000,-0.044260,0.011485,0.027307
itemCommentStatus,0.020401,1.000000,-0.135097,-0.124360,-0.027678,-0.002126,-0.060732,-0.016714,-0.043256,0.023572,0.047079,0.482105,0.038877
duetEnabled,0.038341,-0.135097,1.000000,0.856875,0.010968,0.061653,-0.048290,-0.291476,0.038070,-0.108077,-0.024119,-0.080651,-0.257226
stitchEnabled,0.018964,-0.124360,0.856875,1.000000,0.025391,0.057524,-0.027155,-0.262384,0.044412,-0.076170,-0.056915,-0.069121,-0.221372
date,0.034735,-0.027678,0.010968,0.025391,1.000000,-0.027404,0.024632,-0.017617,0.048776,0.005070,-0.034133,0.001903,-0.011219
video.height,0.051241,-0.002126,0.061653,0.057524,-0.027404,1.000000,0.041422,0.051118,0.135514,0.049152,-0.076350,-0.011179,-0.022158
video.width,0.011305,-0.060732,-0.048290,-0.027155,0.024632,0.041422,1.000000,0.085407,0.091115,-0.011807,-0.084951,-0.146245,-0.011672
video.duration,0.003746,-0.016714,-0.291476,-0.262384,-0.017617,0.051118,0.085407,1.000000,-0.140889,0.250283,-0.056691,-0.010840,0.048992
video.bitrate,-0.044217,-0.043256,0.038070,0.044412,0.048776,0.135514,0.091115,-0.140889,1.000000,-0.078964,-0.068969,0.013553,0.011946
author.verified,0.029000,0.023572,-0.108077,-0.076170,0.005070,0.049152,-0.011807,0.250283,-0.078964,1.000000,-0.073078,-0.022082,0.005105


In [199]:
video_stats.corr().dropna(how='all').dropna(axis=1).iloc[:,13:]

,author.stitchSetting,music.original,music.duration,stats.diggCount,stats.shareCount,stats.commentCount,stats.playCount,authorStats.followingCount,authorStats.followerCount,authorStats.heartCount,authorStats.videoCount,authorStats.diggCount,authorStats.heart
createTime,0.037493,0.043481,-0.032440,-0.065056,-0.027624,-0.110219,-0.093826,-0.012992,-0.026108,0.010104,0.016801,-0.011553,0.010104
itemCommentStatus,0.118311,-0.068479,0.017449,-0.051716,-0.057452,-0.072083,0.026602,-0.002773,-0.033443,-0.023379,0.006973,-0.034420,-0.023379
duetEnabled,-0.226485,0.056993,-0.230185,-0.013078,-0.051917,-0.064206,-0.063352,-0.028340,-0.000891,-0.022772,-0.010109,0.110071,-0.022772
stitchEnabled,-0.212381,0.044485,-0.201372,0.019367,-0.033021,0.007071,-0.042161,-0.037492,0.023560,-0.006150,0.016225,0.133532,-0.006150
date,-0.014819,-0.002216,-0.009365,-0.051988,-0.037471,-0.045218,-0.068707,0.037778,0.011307,0.023174,0.012910,0.040054,0.023174
video.height,-0.006394,-0.046967,0.054391,0.014415,-0.027889,-0.047641,-0.013546,0.012678,-0.001077,-0.008773,0.017921,0.032352,-0.008773
video.width,-0.008334,0.013727,0.050019,-0.063737,-0.071425,-0.077340,-0.120413,-0.037400,0.020108,0.020645,0.005952,-0.006446,0.020645
video.duration,-0.015046,-0.076356,0.710838,0.017140,0.172368,-0.004333,0.026158,-0.050689,0.102608,0.037440,0.070995,-0.099460,0.037440
video.bitrate,0.011256,-0.064198,-0.092073,0.141876,0.009401,0.095498,0.115704,0.069646,0.041387,-0.037203,-0.077902,0.125241,-0.037203
author.verified,-0.075039,0.027158,0.152815,-0.014814,0.084682,-0.000603,0.006422,-0.027112,0.447674,0.313661,0.444338,-0.079128,0.313661


In [200]:
video_stats.describe().iloc[:,:13]

,createTime,itemCommentStatus,date,video.height,video.width,video.duration,video.bitrate,author.relation,author.commentSetting,author.duetSetting,author.stitchSetting,music.duration,stats.diggCount
count,1.563000e+03,1563.000000,1.563000e+03,1563.000000,1563.000000,1563.000000,1.562000e+03,1563.0,1563.000000,1563.000000,1563.000000,1563.000000,1.563000e+03
mean,1.645525e+09,0.047985,9.040339e+06,995.294946,572.418426,26.877159,1.034832e+06,0.0,0.012156,0.163148,0.191299,38.621881,7.346999e+05
std,2.184547e+06,0.376487,1.512833e+06,130.472408,59.005514,30.805079,6.934283e+05,0.0,0.187252,0.645777,0.699278,41.938934,9.166033e+05
min,1.607880e+09,0.000000,7.032022e+06,0.000000,0.000000,0.000000,1.505270e+05,0.0,0.000000,0.000000,0.000000,5.000000,1.000000e+00
25%,1.645724e+09,0.000000,8.032022e+06,1024.000000,576.000000,10.000000,5.786930e+05,0.0,0.000000,0.000000,0.000000,14.000000,2.469500e+05
50%,1.645857e+09,0.000000,9.032022e+06,1024.000000,576.000000,15.000000,8.420870e+05,0.0,0.000000,0.000000,0.000000,26.000000,4.038000e+05
75%,1.646061e+09,0.000000,1.003202e+07,1024.000000,576.000000,28.000000,1.267352e+06,0.0,0.000000,0.000000,0.000000,59.500000,8.450000e+05
max,1.646955e+09,3.000000,1.103202e+07,1280.000000,1280.000000,404.000000,4.339947e+06,0.0,3.000000,3.000000,3.000000,475.000000,7.500000e+06


In [201]:
video_stats.describe().iloc[:,13:]

,stats.shareCount,stats.commentCount,stats.playCount,authorStats.followingCount,authorStats.followerCount,authorStats.heartCount,authorStats.videoCount,authorStats.diggCount,authorStats.heart
count,1563.000000,1563.000000,1.563000e+03,1563.00000,1.563000e+03,1.563000e+03,1563.000000,1563.000000,1.563000e+03
mean,19433.156110,12553.751759,6.447945e+06,576.46833,3.960896e+05,1.252589e+07,337.113244,15643.838772,1.252589e+07
std,31432.199154,22211.286944,7.346850e+06,1454.75129,1.164650e+06,5.983273e+07,670.973191,36558.021038,5.983273e+07
min,0.000000,0.000000,4.400000e+01,0.00000,1.000000e+00,1.900000e+01,1.000000,0.000000,1.900000e+01
25%,2886.500000,2747.000000,2.100000e+06,22.50000,1.070000e+04,4.736000e+05,22.000000,417.000000,4.736000e+05
50%,7663.000000,6234.000000,4.100000e+06,120.00000,4.770000e+04,1.600000e+06,84.000000,2576.000000,1.600000e+06
75%,21100.000000,14900.000000,8.000000e+06,473.00000,2.417000e+05,5.700000e+06,322.500000,14800.000000,5.700000e+06
max,233000.000000,329200.000000,5.430000e+07,10000.00000,2.190000e+07,2.000000e+09,6902.000000,322600.000000,2.000000e+09


In [203]:
user_stats.describe()

,date,user.createTime,user.relation,user.bioLink.risk,user.commentSetting,user.duetSetting,user.stitchSetting,stats.followerCount,stats.followingCount,stats.heart,stats.heartCount,stats.videoCount,stats.diggCount
count,1.562000e+03,1.555000e+03,1555.0,533.000000,1555.000000,1555.000000,1555.000000,1.555000e+03,1555.000000,1.555000e+03,1.555000e+03,1555.000000,1555.000000
mean,9.040345e+06,1.592694e+09,0.0,1.932458,0.012219,0.163987,0.192283,3.985359e+05,578.423151,1.258509e+07,1.258509e+07,338.533119,365.515756
std,1.513317e+06,4.423104e+07,0.0,1.507854,0.187732,0.647331,0.700940,1.168873e+06,1458.174804,5.998075e+07,5.998075e+07,672.392840,3000.385750
min,7.032022e+06,1.437036e+09,0.0,0.000000,0.000000,0.000000,0.000000,1.000000e+00,0.000000,1.900000e+01,1.900000e+01,1.000000,0.000000
25%,8.032022e+06,1.572444e+09,0.0,0.000000,0.000000,0.000000,0.000000,1.085000e+04,22.500000,4.755000e+05,4.755000e+05,22.000000,0.000000
50%,9.032022e+06,1.592870e+09,0.0,3.000000,0.000000,0.000000,0.000000,4.810000e+04,120.000000,1.600000e+06,1.600000e+06,84.000000,0.000000
75%,1.003202e+07,1.628988e+09,0.0,3.000000,0.000000,0.000000,0.000000,2.421000e+05,475.000000,5.900000e+06,5.900000e+06,324.000000,0.000000
max,1.103202e+07,1.646904e+09,0.0,5.000000,3.000000,3.000000,3.000000,2.190000e+07,10000.000000,2.000000e+09,2.000000e+09,6902.000000,50700.000000


## PCA
- used imcremental PCA because of the size of the dataset

In [274]:
v_cols = ['video.duration', 'music.duration', 'stats.diggCount', 'stats.shareCount', 'stats.commentCount', 
          'stats.playCount', 'authorStats.followingCount', 'authorStats.followerCount', 'authorStats.heartCount', 
          'authorStats.videoCount', 'authorStats.diggCount']
X = video_stats[v_cols]
X = StandardScaler().fit_transform(X)

pca = IncrementalPCA(n_components=6, batch_size = 100)

X_sparse = sparse.csr_matrix(X)
X_transformed = pca.fit_transform(X_sparse)
X_transformed.shape

print(pca.explained_variance_ratio_, sum(pca.explained_variance_ratio_))
print(pca.singular_values_)

[0.26853893 0.21915507 0.15920727 0.10581459 0.06395606 0.03644098] 0.8531129049368521
[67.94843524 61.38349153 52.31874078 42.65290356 33.16016617 25.0305783 ]


In [286]:
PC_nums = []
for i in range(6):
    PC_nums.append(f'PC {i+1}')

pd.DataFrame(pca.components_,columns=v_cols,index = PC_nums)

,video.duration,music.duration,stats.diggCount,stats.shareCount,stats.commentCount,stats.playCount,authorStats.followingCount,authorStats.followerCount,authorStats.heartCount,authorStats.videoCount,authorStats.diggCount
PC 1,0.070957,0.034990,0.521442,0.457742,0.473858,0.523268,-0.079418,0.047453,0.004985,-0.029784,-0.067003
PC 2,0.106569,0.077583,0.005566,0.013252,0.006319,-0.017587,0.232216,0.563312,0.567879,0.521292,0.128807
PC 3,0.676800,0.665788,-0.090985,0.060061,-0.095259,-0.062630,-0.160591,-0.013976,-0.064334,-0.031185,-0.206616
PC 4,0.140847,0.203834,0.072384,0.056919,0.069385,-0.018612,0.585465,-0.231268,-0.185417,-0.045095,0.701646
PC 5,-0.022340,-0.036194,-0.122144,0.116956,0.148312,-0.052992,0.616449,-0.239814,-0.164763,0.320520,-0.614320
PC 6,-0.036824,0.074983,-0.291823,-0.038325,0.766749,-0.402086,-0.002720,0.115879,0.174617,-0.335976,-0.002649


In [270]:
u_cols = ['stats.followerCount', 'stats.followingCount', 'stats.heart', 'stats.videoCount', 'stats.diggCount']
X2 = user_stats[u_cols]
X2 = X2.dropna()

X2 = StandardScaler().fit_transform(X2)

pca2 = IncrementalPCA(n_components=3, batch_size = 200)

X2_sparse = sparse.csr_matrix(X2)
X2_transformed = pca2.fit_transform(X2_sparse)
X2_transformed.shape

print(pca2.explained_variance_ratio_, sum(pca2.explained_variance_ratio_))
print(pca2.singular_values_)

[0.47392934 0.22660795 0.15794287] 0.8584801515546443
[60.70255832 41.97471594 35.04291388]


In [273]:
PC_nums = []
for i in range(3):
    PC_nums.append(f'PC {i+1}')

pd.DataFrame(pca2.components_,columns=u_cols,index = PC_nums)

,stats.followerCount,stats.followingCount,stats.heart,stats.videoCount,stats.diggCount
PC 1,0.573833,0.221102,0.581701,0.532340,0.008240
PC 2,-0.178998,0.585503,-0.115609,0.064030,0.779539
PC 3,-0.259663,0.699650,-0.158970,0.172661,-0.622881


## F Regression

In [299]:
for i in range(len(v_cols)):
    print(f'y = {v_cols[i]} \nf statistic: {f_regression(X, X[:,i])[0]} \np values: {f_regression(X, X[:,i])[1]}')

y = video.duration 
f statistic: [2.51075679e+17 1.59438677e+03 4.58729729e-01 4.77982917e+01
 2.93123643e-02 1.06885050e+00 4.02112470e+00 1.66097454e+01
 2.19120399e+00 7.90771091e+00 1.55962514e+01] 
p values: [0.00000000e+000 7.82596726e-241 4.98318596e-001 6.86340892e-012
 8.64081982e-001 3.01365374e-001 4.51060890e-002 4.82231291e-005
 1.39002572e-001 4.98403096e-003 8.19013078e-005]
y = music.duration 
f statistic: [1.59438677e+03 4.68674601e+17 5.83607858e-01 1.18998021e+01
 1.05298514e+00 2.64004389e-01 1.53456619e+00 3.15246925e+00
 5.12066213e-02 2.12682871e+00 4.07879502e+00] 
p values: [7.82596726e-241 0.00000000e+000 4.45017072e-001 5.76341070e-004
 3.04979567e-001 6.07455058e-001 2.15614672e-001 7.60069535e-002
 8.21006434e-001 1.44941105e-001 4.35952046e-002]
y = stats.diggCount 
f statistic: [ 4.58729729e-01  5.83607858e-01 -1.75752975e+18  8.34991238e+02
  1.19446083e+03  3.10170479e+03  8.90878752e+00  7.10994436e+00
  2.91262694e-01  2.59058999e+00  2.23617994e-01] 

In [297]:
for i in range(len(u_cols)):
    print(f'y = {u_cols[i]} \nf statistics: {f_regression(X2, X2[:,i])[0]}, \np values: {f_regression(X2, X2[:,i])[1]}')

y = stats.followerCount: 
f statistics: [           inf 2.39690339e+01 2.61368855e+03 7.54904998e+02
 2.02948706e+00], 
p values: [0.00000000e+000 1.08094325e-006 0.00000000e+000 9.00096517e-136
 1.54473581e-001]
y = stats.followingCount: 
f statistics: [ 2.39690339e+01 -1.74852256e+18  5.18434809e+01  1.01484540e+02
  3.79441657e+01], 
p values: [1.08094325e-06 1.00000000e+00 9.32853699e-13 3.64150335e-23
 9.25449706e-10]
y = stats.heart: 
f statistics: [2.61368855e+03 5.18434809e+01 2.33136341e+18 7.86285890e+02
 6.36625890e-01], 
p values: [0.00000000e+000 9.32853699e-013 0.00000000e+000 2.47706750e-140
 4.25057242e-001]
y = stats.videoCount: 
f statistics: [ 7.54904998e+02  1.01484540e+02  7.86285890e+02 -1.74852256e+18
  4.29263962e-01], 
p values: [9.00096517e-136 3.64150335e-023 2.47706750e-140 1.00000000e+000
 5.12447365e-001]
y = stats.diggCount: 
f statistics: [2.02948706e+00 3.79441657e+01 6.36625890e-01 4.29263962e-01
 6.99409022e+17], 
p values: [1.54473581e-01 9.25449706e

/Users/arielsilbert/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:306: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/Users/arielsilbert/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:306: RuntimeWarning: divide by zero encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom


## Word vectorization